# Circuit Toolkit notebook tour

This notebook walks through the new notebook-first helpers: the `CircuitBuilder` DSL,
connectivity tables, and interactive visualization widgets.

## Setup

Install the visualization extra if you have not already:

```bash
pip install circuit-toolkit[viz]
```

All other dependencies used below come with that extra.

In [1]:
# Core imports for the walkthrough
import importlib

import numpy as np
import spicelab.viz.notebook as viz
import xarray as xr
from spicelab.dsl import CircuitBuilder
from spicelab.viz.notebook import connectivity_widget, dataset_plot_widget

importlib.reload(viz)

<module 'spicelab.viz.notebook' from '/Users/lgili/Documents/01 - Codes/01 - Github/circuit_toolkit/spicelab/viz/notebook.py'>

## Build a circuit with the DSL

Define nets once and drop components with auto-numbered references.

In [2]:
builder = CircuitBuilder("rc_notebook_demo")
vin = builder.net("VIN")
vout = builder.net("VOUT")

builder.vdc(vin, "0", value="5V")
builder.resistor(vin, vout, value="1k")
builder.capacitor(vout, "0", value="100n")
builder.resistor(vout, "0", value="10k", ref="RLOAD")

circuit = builder.circuit
circuit

Circuit(name='rc_notebook_demo', _net_ids={}, _port_to_net={Port(owner=<Vdc V1 value='5V'>, name='p', role=<PortRole.POSITIVE: 1>): Net(name='VIN'), Port(owner=<Vdc V1 value='5V'>, name='n', role=<PortRole.NEGATIVE: 2>): Net(name='0'), Port(owner=<Resistor R1 value='1k'>, name='a', role=<PortRole.POSITIVE: 1>): Net(name='VIN'), Port(owner=<Resistor R1 value='1k'>, name='b', role=<PortRole.NEGATIVE: 2>): Net(name='VOUT'), Port(owner=<Capacitor C1 value='100n'>, name='a', role=<PortRole.POSITIVE: 1>): Net(name='VOUT'), Port(owner=<Capacitor C1 value='100n'>, name='b', role=<PortRole.NEGATIVE: 2>): Net(name='0'), Port(owner=<Resistor RLOAD value='10k'>, name='a', role=<PortRole.POSITIVE: 1>): Net(name='VOUT'), Port(owner=<Resistor RLOAD value='10k'>, name='b', role=<PortRole.NEGATIVE: 2>): Net(name='0')}, _components=[<Vdc V1 value='5V'>, <Resistor R1 value='1k'>, <Capacitor C1 value='100n'>, <Resistor RLOAD value='10k'>], _directives=[], _subckt_defs={}, _subckt_instances=[])

## Inspect the generated netlist

In [3]:
print(circuit.build_netlist())

* rc_notebook_demo
VV1 VIN 0 5V
RR1 VIN VOUT 1k
CC1 VOUT 0 100n
RRLOAD VOUT 0 10k
.end



## Summary table

The fixed-width summary mirrors `Circuit.summary()` but renders cleanly in plain text.

In [4]:
print(circuit.summary_table())

  component  type       port  net 
  ---------  ---------  ----  ----
  C1         Capacitor  a     VOUT
  C1         Capacitor  b     0   
  R1         Resistor   a     VIN 
  R1         Resistor   b     VOUT
  RLOAD      Resistor   a     VOUT
  RLOAD      Resistor   b     0   
  V1         Vdc        p     VIN 
  V1         Vdc        n     0   


## Connectivity dataframe

A pandas table is handy for quick filtering or joins with measurement data.

In [5]:
circuit.connectivity_dataframe()

,component,type,port,net
0,C1,Capacitor,a,VOUT
1,C1,Capacitor,b,0
2,R1,Resistor,a,VIN
3,R1,Resistor,b,VOUT
4,RLOAD,Resistor,a,VOUT
5,RLOAD,Resistor,b,0
6,V1,Vdc,p,VIN
7,V1,Vdc,n,0


## Interactive widget

Browse component connectivity with a dropdown + HTML preview.

In [6]:
connectivity_widget(circuit)

## Create a dataset to plot

Here we mock a low-pass filter frequency response using NumPy and xarray.

In [7]:
freq = np.logspace(2, 5, 60)
fc = 1e4
gain = 1 / np.sqrt(1 + (freq / fc) ** 2)
phase = -np.arctan(freq / fc)

dataset = xr.Dataset(
    {
        "gain": ("freq", gain),
        "phase": ("freq", phase),
    },
    coords={"freq": freq},
    attrs={"description": "RC low-pass frequency response"},
)
dataset

<xarray.Dataset> Size: 1kB
Dimensions:  (freq: 60)
Coordinates:
  * freq     (freq) float64 480B 100.0 112.4 126.4 ... 7.912e+04 8.895e+04 1e+05
Data variables:
    gain     (freq) float64 480B 1.0 0.9999 0.9999 ... 0.1254 0.1117 0.0995
    phase    (freq) float64 480B -0.01 -0.01124 -0.01264 ... -1.459 -1.471
Attributes:
    description:  RC low-pass frequency response

## Plot interactively

The dataset widget provides dropdown selectors and a live Plotly FigureWidget.

In [8]:
dataset_plot_widget(dataset)

## Monte Carlo dataset

Simulate multiple trials and explore them with the same widget. Trials become
dropdown selectors for the extra dimension.


In [9]:
rng = np.random.default_rng(seed=42)
trials = np.arange(1, 11)
mc_freq = np.logspace(2, 5, 80)

nominal_gain = 1 / np.sqrt(1 + (mc_freq / fc) ** 2)
noise = rng.normal(scale=0.05, size=(len(trials), len(mc_freq)))
mc_gain = nominal_gain * (1 + noise)

mc_dataset = xr.Dataset(
    {
        "gain": (("trial", "freq"), mc_gain),
    },
    coords={"trial": trials, "freq": mc_freq},
    attrs={
        "description": "Monte Carlo sweep for RC low-pass",
        "unit": "V/V",
    },
)

mc_dataset

<xarray.Dataset> Size: 7kB
Dimensions:  (trial: 10, freq: 80)
Coordinates:
  * trial    (trial) int64 80B 1 2 3 4 5 6 7 8 9 10
  * freq     (freq) float64 640B 100.0 109.1 119.1 ... 8.396e+04 9.163e+04 1e+05
Data variables:
    gain     (trial, freq) float64 6kB 1.015 0.9479 1.037 ... 0.1141 0.0997
Attributes:
    description:  Monte Carlo sweep for RC low-pass
    unit:         V/V

In [10]:
dataset_plot_widget(mc_dataset)

## Where to go next

Try extending the circuit, running AC analyses via the engines, or pairing the widgets
with Monte Carlo datasets generated by the higher-level workflows.